In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [2]:
import xarray as xr
import numpy as np
import pandas as pd

import ecephys_analyses as ea 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [3]:
xr.set_options(keep_attrs=True)

In [4]:
def normalize_spws(spws, norm_spws):
    _spws = spws.copy()
    for col in ['sink_amplitude', 'duration', 'sink_integral']:
        _spws[col] = _spws[col] / norm_spws[col].mean()
        
    return _spws

In [9]:
def summarize(subject, condition, spws):
    return dict(subject=subject, condition=condition, amplitude=spws.sink_amplitude.sum(), integral=spws.sink_integral.sum(), duration=spws.duration.sum())

In [10]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
subject_dfs = list()
for subject in subjects:
    norm_hyp, norm_spws = ea.load_baseline_light_period_nrem(subject)
    dep_hyp, dep_spws = ea.load_deprivation(subject)
    early_hyp, early_spws = ea.load_early_recovery_nrem(subject)
    late_hyp, late_spws = ea.load_late_recovery_nrem(subject)
    rec_2h_hyp, rec_2h_spws = ea.load_first2h_recovery_nrem(subject)
    rec_6h_hyp, rec_6h_spws = ea.load_recovery_nrem(subject)
    bsl_2h_hyp, bsl_2h_spws = ea.load_first2h_recovery_match_nrem(subject, rec_2h_hyp)
    
    _norm_spws = norm_spws.copy()

    norm_spws = normalize_spws(norm_spws, _norm_spws)
    dep_spws = normalize_spws(dep_spws, _norm_spws)
    early_spws = normalize_spws(early_spws, _norm_spws)
    late_spws = normalize_spws(late_spws, _norm_spws)
    rec_2h_spws = normalize_spws(rec_2h_spws, _norm_spws)
    rec_6h_spws = normalize_spws(rec_6h_spws, _norm_spws)
    bsl_2h_spws = normalize_spws(bsl_2h_spws, _norm_spws)

    subject_df = pd.DataFrame.from_dict(
        [
            summarize(subject, "baseline-light-period-nrem", norm_spws),
            summarize(subject, "recovery-first2h-match", bsl_2h_spws),
            summarize(subject, "recovery-first2h", rec_2h_spws),
            summarize(subject, "recovery-first1h", early_spws),
            summarize(subject, "recovery-last1h", late_spws),
            summarize(subject, "recovery-full", rec_6h_spws),
            summarize(subject, "deprivation", dep_spws)
        ]
    )

    subject_dfs.append(subject_df)

In [11]:
df = pd.concat(subject_dfs)
df

,subject,condition,amplitude,integral,duration
0,Segundo,baseline-light-period-nrem,19963.000000,19963.000000,19963.000000
1,Segundo,recovery-first2h-match,5180.742121,5150.010821,5126.779743
2,Segundo,recovery-first2h,7877.581258,7943.564865,7584.458402
3,Segundo,recovery-first1h,4269.364502,4239.341133,4000.267869
4,Segundo,recovery-last1h,3006.722631,2935.376435,2855.832188
5,Segundo,recovery-full,11705.087429,11711.159021,11253.980291
6,Segundo,deprivation,9418.345504,7970.698295,7512.935632
0,Valentino,baseline-light-period-nrem,13497.000000,13497.000000,13497.000000
1,Valentino,recovery-first2h-match,4279.406980,4244.991974,4184.950941
2,Valentino,recovery-first2h,6454.212066,6760.603584,6446.758132


In [12]:
df.to_csv('total-spw-energy.csv', index=False)